In [1]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
# Load predictions
fnam = "/home/jbassham/jack/thesis-rough/data/sh/outputs/cnn/weighted/CNNPreds_sh_1992_2020.npz"
data = np.load(fnam, allow_pickle = True)

y_pred = data['y_pred']
y_true = data['y_true']

In [3]:
u_pred=  y_pred[:,0,:,:]
u_true = y_true[:,0,:,:]

v_pred = y_pred[:,1,:,:]
v_true = y_pred[:,1,:,:]

In [4]:
# load latitude longitude grid for plotting
fnam = "/home/jbassham/jack/thesis-rough/data/sh/inputs/lat_lon_sh_1992_2020.npz"
data = np.load(fnam, allow_pickle = True)

lat = data['lat']
lon = data['lon']

In [5]:
# Load CNN data split indices and land mask
fnam = "/home/jbassham/jack/thesis-rough/data/sh/cnn-inputs/indices_land_sh_1992_2020.npz"
data = np.load(fnam, allow_pickle = True)

train_idx = data['train_idx']
val_idx = data['val_idx']
test_idx = data['test_idx']
land_mask = data['land_mask']

In [6]:
# Load uncertainty, ice velocities
fnam = "/home/jbassham/jack/thesis-rough/data/sh/inputs/inputs_normalized_sh_1992_2020.npz"
data = np.load(fnam, allow_pickle=True)
rt = data['rtn']
uit = data['uitn']
vit = data['vitn']

In [7]:
# Crop uncertainty, ice velocities to test indices
r_test = rt[test_idx,:,:]
ui_test = uit[test_idx,:,:] 
vi_test = vit[test_idx,:,:]

In [ ]:
def correlation(pred, true):

    """
    Pearson Correlation
    """

    predbar = np.nanmean(pred, axis = 0) # mean predicted
    truebar = np.nanmean(true, axis = 0) # mean true

    covariance = np.nansum((pred - predbar) * (true - truebar), axis = 0) # covariance between predicted and true
    
    stdpred = np.sqrt(np.nansum((pred - predbar)**2), axis = 0) # standard deviation predited
    stdtrue = np.sqrt(np.nansum((true - truebar)**2), axis = 0) # standard deviation true

    correlation = covariance / (stdpred * stdtrue)

    return correlation

In [ ]:
def weighted_correlation(pred, true, r, epsilon = 1e-4):

    """
    Weighted Pearson Correlation referenced from:
    https://www.air.org/sites/default/files/2021-06/Weighted-and-Unweighted-Correlation-Methods-Large-Scale-Educational-Assessment-April-2018.pdf
    
    """

    w = 1 / (r + epsilon)

    def weighted_mean(x, w):
        return np.nansum(w * w, axis = 0) / np.nansum(w, axis = 0)

    predbar = weighted_mean(pred, axis = 0) # weighted mean predicted
    truebar = weighted_mean(true, axis = 0) # weighted mean true

    weighted_cov = np.nansum( w * (pred - predbar) * (true - truebar), axis = 0) # weighted covariance between predicted and true
    
    weighted_stdpred = np.sqrt(np.nansum(w * (pred - predbar)**2), axis = 0) # weighted standard deviation predited
    weighted_stdtrue = np.sqrt(np.nansum(w * (true - truebar)**2), axis = 0) # weighted standard deviation true

    correlation = weighted_cov / (weighted_stdpred * weighted_stdtrue)

    return correlation

In [ ]:
def skill(pred, true, epsilon = 1e-4):
    # NOTE excluding epsilon = 1e-4 from denominator for now

    mse = np.nanmean((true - pred)**2, axis = 0) # mean square error
    # NOTE above is not equivalent to np.nanvar(true-pred), which excludes bias term
    # MSE = E[(y-x)^2]
    # = (E[y-x])^2 + Var(y-x)
    # = bias^2 + Var(y-x)
    # Can prove the above

    truebar = np.nanmean(true, axis = 0) # mean true

    vartrue = np.nanmean((true - truebar)**2, axis = 0) # variance in true
    # NOTE above is equivalent to np.nanvar()

    skill = 1 - mse / vartrue

    return skill

In [ ]:
def weighted_skill(pred, true, r, epsilon = 1e-4):
    # NOTE excluding epsilon = 1e-4 from denominator for now
    # NOTE including epsilon = 1e-4 in the weights in case of uncertainty r ~ 0

    w = 1 / (r + epsilon)

    mse = np.nanmean(( w * (true - pred))**2, axis = 0) # mean square error
    # NOTE above is not equivalent to np.nanvar(true-pred), which excludes bias term

    truebar = np.nanmean(true, axis = 0) # mean true

    vartrue = np.nanmean(( w * (true - truebar))**2, axis = 0) # variance in true
    # NOTE above is equivalent to np.nanvar()

    weighted_skill = 1 - mse / vartrue

    return weighted_skill